In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import squareform, cdist, pdist

import ovwmols

In [2]:
dfs = ovwmols.readFiles(['sample/test1.xyz','sample/test2.xyz'], 'XYZ')
#test1.xyzでそれぞれphenolのC2,C1,O,H
refAtomIndexes = [0,1,11,10]

In [3]:
dfs[0]

,elementSymbol,x,y,z
0,C,0.080457,-0.863302,0.000218
1,C,1.458946,-0.625066,0.000011
2,C,1.948032,0.685583,-0.000158
3,C,1.049990,1.757404,-0.000099
4,C,-0.327549,1.527620,0.000120
5,C,-0.808717,0.212371,0.000265
6,H,-0.276298,-1.888352,0.000345
7,H,3.020953,0.868020,-0.000329
8,H,1.435398,2.773253,-0.000223
9,H,-1.021912,2.362133,0.000171


In [4]:
dfs[1]

,elementSymbol,x,y,z
0,C,1.242635,1.492466,0.000579
1,C,-0.012837,0.878221,0.001362
2,C,-0.102538,-0.521387,0.000770
3,C,1.066458,-1.294624,-0.000390
4,C,2.312751,-0.668607,-0.001069
5,C,2.411912,0.727578,-0.000634
6,H,3.384510,1.209893,-0.001210
7,O,-1.296117,-1.181977,0.001276
8,H,-2.043843,-0.554664,0.001191
9,H,3.213234,-1.276807,-0.001980


In [5]:
#後の原子の対応関係を調べる際に楽をするために元素記号でソートしておく
df0 = dfs[0].sort_values('elementSymbol')
df1 = dfs[1].sort_values('elementSymbol')
df0

,elementSymbol,x,y,z
0,C,0.080457,-0.863302,0.000218
1,C,1.458946,-0.625066,0.000011
2,C,1.948032,0.685583,-0.000158
3,C,1.049990,1.757404,-0.000099
4,C,-0.327549,1.527620,0.000120
5,C,-0.808717,0.212371,0.000265
6,H,-0.276298,-1.888352,0.000345
7,H,3.020953,0.868020,-0.000329
8,H,1.435398,2.773253,-0.000223
9,H,-1.021912,2.362133,0.000171


In [6]:
df1

,elementSymbol,x,y,z
0,C,1.242635,1.492466,0.000579
1,C,-0.012837,0.878221,0.001362
2,C,-0.102538,-0.521387,0.000770
3,C,1.066458,-1.294624,-0.000390
4,C,2.312751,-0.668607,-0.001069
5,C,2.411912,0.727578,-0.000634
6,H,3.384510,1.209893,-0.001210
8,H,-2.043843,-0.554664,0.001191
9,H,3.213234,-1.276807,-0.001980
10,H,0.978066,-2.376464,-0.000788


In [7]:
#元素記号のソートの結果をrefAtomIndexesに反映する
#内容は変わらないが、df0のデータの並び順に合うようになる
refAtomIndexes = [i for i in df0.index if i in refAtomIndexes]
#df0のインデックスの順番は変わっているのでlocで取得
#refAtomIndexesを並び替えたので、df0refsも元素記号順になっている
df0refs =df0.loc[refAtomIndexes]

print(refAtomIndexes)
df0refs

[0, 1, 11, 10]


,elementSymbol,x,y,z
0,C,0.080457,-0.863302,0.000218
1,C,1.458946,-0.625066,0.000011
11,H,3.210430,-1.433662,-0.000200
10,O,2.288102,-1.720862,-0.000047


In [8]:
#
#df0refsとdf1内の原子の数を元素毎に求める
#後でdf0refsとdf1の原子間距離の比較をする際に、データを複製する必要があるのでその数を求める
#また、df0refs内に1つしかない元素を探すのも目的
#indexが元素記号、値が各元素の原子数のSeriesになる
#value_counts()では多い順に元素が並んでしまうため、sort_index()で元素記号順に並べ直す
df0refs_numEachElements = df0refs['elementSymbol'].value_counts().sort_index()
df1_numEachElements = df1['elementSymbol'].value_counts().sort_index()

# ref内で1つしかない元素を探す
onlyone_ref_symbolList = df0refs_numEachElements[df0refs_numEachElements==1].index.tolist()

print(df0refs_numEachElements)
print(df1_numEachElements)
print(onlyone_ref_symbolList)

C    2
H    1
O    1
Name: elementSymbol, dtype: int64
C    6
H    8
O    2
Name: elementSymbol, dtype: int64
['H', 'O']


In [9]:
df1_numEachElements.loc[onlyone_ref_symbolList]

H    8
O    2
Name: elementSymbol, dtype: int64

In [10]:
#onlyone_ref_symbolListに含まれる元素の内、df1内でも少ない元素を選定
ele = df1_numEachElements.loc[onlyone_ref_symbolList].idxmin()
ele

'O'

In [11]:
#df1中の各原子とele候補(df1内で1つだけとは限らない)の距離行列を計算
#df0refsの距離行列と比較することでdf1の原子を絞る
#
#df0refs内でのeleとの距離行列を計算
df0refs_eleIndex = np.where(df0refs['elementSymbol']==ele)
df0refs_dis = squareform(pdist(df0refs[['x','y','z']]))[df0refs_eleIndex]
#df1内でのele(候補)との距離行列を計算
df1_dis = cdist(df1[df1['elementSymbol']==ele][['x','y','z']], df1[['x','y','z']])
#
df0refs_dis

array([[2.36835506, 1.37414284, 0.9660087 , 0.        ]])

In [12]:
df1_dis

array([[3.68753408, 2.42718425, 1.36418849, 2.36525955, 3.64519993,
        4.17083721, 5.25635971, 0.97602038, 4.51034919, 2.56879578,
        2.6845307 , 4.56993013, 3.36628831, 3.36471049, 0.        ,
        2.83903112],
       [4.93656241, 3.59918163, 3.63193717, 4.98744471, 6.01622882,
        6.00759503, 7.01257554, 1.86650078, 7.02971372, 5.39348426,
        2.84707565, 5.31985713, 0.965995  , 0.96599867, 2.83903112,
        0.        ]])

In [13]:
df0refs

,elementSymbol,x,y,z
0,C,0.080457,-0.863302,0.000218
1,C,1.458946,-0.625066,0.000011
11,H,3.210430,-1.433662,-0.000200
10,O,2.288102,-1.720862,-0.000047


In [14]:
df1

,elementSymbol,x,y,z
0,C,1.242635,1.492466,0.000579
1,C,-0.012837,0.878221,0.001362
2,C,-0.102538,-0.521387,0.000770
3,C,1.066458,-1.294624,-0.000390
4,C,2.312751,-0.668607,-0.001069
5,C,2.411912,0.727578,-0.000634
6,H,3.384510,1.209893,-0.001210
8,H,-2.043843,-0.554664,0.001191
9,H,3.213234,-1.276807,-0.001980
10,H,0.978066,-2.376464,-0.000788


In [15]:
#df0refs_disとdf1_disの原子対同士を比較するため、
#それぞれを列・行方向に複製し、差をとることで全組合せを比較
#→ゼロに近い=原子対がdf0とdf1とで対応
#まずはdf0refs_disについて
df0refs_disrepeat = np.repeat(df0refs_dis,
                              np.repeat(df1_numEachElements,df0refs_numEachElements), #[6,6,8,2]
                             )
df0refs_disrepeat = np.tile(df0refs_disrepeat,(2,1))
df0refs_disrepeat

array([[2.36835506, 2.36835506, 2.36835506, 2.36835506, 2.36835506,
        2.36835506, 1.37414284, 1.37414284, 1.37414284, 1.37414284,
        1.37414284, 1.37414284, 0.9660087 , 0.9660087 , 0.9660087 ,
        0.9660087 , 0.9660087 , 0.9660087 , 0.9660087 , 0.9660087 ,
        0.        , 0.        ],
       [2.36835506, 2.36835506, 2.36835506, 2.36835506, 2.36835506,
        2.36835506, 1.37414284, 1.37414284, 1.37414284, 1.37414284,
        1.37414284, 1.37414284, 0.9660087 , 0.9660087 , 0.9660087 ,
        0.9660087 , 0.9660087 , 0.9660087 , 0.9660087 , 0.9660087 ,
        0.        , 0.        ]])

In [16]:
print(df0refs_numEachElements)
print(df1_numEachElements)

C    2
H    1
O    1
Name: elementSymbol, dtype: int64
C    6
H    8
O    2
Name: elementSymbol, dtype: int64


In [17]:
#df1_disについて
df1_dis

array([[3.68753408, 2.42718425, 1.36418849, 2.36525955, 3.64519993,
        4.17083721, 5.25635971, 0.97602038, 4.51034919, 2.56879578,
        2.6845307 , 4.56993013, 3.36628831, 3.36471049, 0.        ,
        2.83903112],
       [4.93656241, 3.59918163, 3.63193717, 4.98744471, 6.01622882,
        6.00759503, 7.01257554, 1.86650078, 7.02971372, 5.39348426,
        2.84707565, 5.31985713, 0.965995  , 0.96599867, 2.83903112,
        0.        ]])

In [18]:
np.split(df1_dis, np.cumsum(df1_numEachElements), axis=1)[:-1]
#それぞれにtileをして、結合してやればいい

[array([[3.68753408, 2.42718425, 1.36418849, 2.36525955, 3.64519993,
         4.17083721],
        [4.93656241, 3.59918163, 3.63193717, 4.98744471, 6.01622882,
         6.00759503]]),
 array([[5.25635971, 0.97602038, 4.51034919, 2.56879578, 2.6845307 ,
         4.56993013, 3.36628831, 3.36471049],
        [7.01257554, 1.86650078, 7.02971372, 5.39348426, 2.84707565,
         5.31985713, 0.965995  , 0.96599867]]),
 array([[0.        , 2.83903112],
        [2.83903112, 0.        ]])]

In [19]:
df1_disrepeat = np.hstack(
    [np.tile(ar, n) for ar, n in zip(
        np.split(df1_dis, np.cumsum(df1_numEachElements), axis=1)[:-1],
        df0refs_numEachElements
    )]
)
df1_disrepeat

array([[3.68753408, 2.42718425, 1.36418849, 2.36525955, 3.64519993,
        4.17083721, 3.68753408, 2.42718425, 1.36418849, 2.36525955,
        3.64519993, 4.17083721, 5.25635971, 0.97602038, 4.51034919,
        2.56879578, 2.6845307 , 4.56993013, 3.36628831, 3.36471049,
        0.        , 2.83903112],
       [4.93656241, 3.59918163, 3.63193717, 4.98744471, 6.01622882,
        6.00759503, 4.93656241, 3.59918163, 3.63193717, 4.98744471,
        6.01622882, 6.00759503, 7.01257554, 1.86650078, 7.02971372,
        5.39348426, 2.84707565, 5.31985713, 0.965995  , 0.96599867,
        2.83903112, 0.        ]])

In [20]:
delta = np.abs(df1_disrepeat - df0refs_disrepeat)
delta

array([[1.31917902e+00, 5.88291905e-02, 1.00416657e+00, 3.09550275e-03,
        1.27684487e+00, 1.80248215e+00, 2.31339123e+00, 1.05304140e+00,
        9.95435272e-03, 9.91116711e-01, 2.27105708e+00, 2.79669437e+00,
        4.29035101e+00, 1.00116808e-02, 3.54434049e+00, 1.60278709e+00,
        1.71852200e+00, 3.60392144e+00, 2.40027961e+00, 2.39870179e+00,
        0.00000000e+00, 2.83903112e+00],
       [2.56820736e+00, 1.23082657e+00, 1.26358211e+00, 2.61908966e+00,
        3.64787376e+00, 3.63923997e+00, 3.56241957e+00, 2.22503879e+00,
        2.25779433e+00, 3.61330187e+00, 4.64208598e+00, 4.63345218e+00,
        6.04656685e+00, 9.00492080e-01, 6.06370503e+00, 4.42747556e+00,
        1.88106696e+00, 4.35384843e+00, 1.36945483e-05, 1.00231864e-05,
        2.83903112e+00, 0.00000000e+00]])

In [21]:
# trueの箇所は原子対が対応していた場所(と思われる)(閾値の関係)
( delta < 0.5 ) & ( delta > 0 )

array([[False,  True, False,  True, False, False, False, False,  True,
        False, False, False, False,  True, False, False, False, False,
        False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
         True,  True, False, False]])

In [22]:
delta == 0

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True]])